In [17]:
from agents import Agent, Runner, trace, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStdio

from dotenv import load_dotenv
import os

from IPython.display import display, Markdown
from datetime import datetime

load_dotenv(override=True)

True

# 3 MCP Servers
To give these powers:
- Memory
- Internet Search
- Live Market Data

## Server 1 - Knowledge-Graph Based Memory

In [6]:
os.makedirs('./memory', exist_ok=True)

params = {
    "command": "npx",
    "args": [
        "-y",
        "mcp-memory-libsql",
    ],
    "env": {
        "LIBSQL_URL": "file:./memory/ed.db",
    }
}

async with MCPServerStdio(params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [7]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversation"
request = "My name is Ed. I'm running a workshop live about AI Agents, and co-presenting with Jon, from the SuperDataScience podcast."

client = AsyncOpenAI(api_key=os.environ['GEMINI_API_KEY'], base_url=os.environ['GEMINI_BASE_URL'])
model = OpenAIChatCompletionsModel('gemini-2.5-flash', client)

In [8]:
async with MCPServerStdio(params) as server:
    agent = Agent(name='agent', instructions=instructions, model=model, mcp_servers=[server])
    
    with trace('conversation'):
        result = await Runner.run(agent, request)
    
    display(Markdown(result.final_output))

That sounds like a very interesting workshop, Ed! Good luck with it. I've stored this information so I can remember it for our future interactions.

In [9]:
question = 'My name is Ed. What do you know about me?'

async with MCPServerStdio(params) as server:
    agent = Agent(name='agent', instructions=instructions, model=model, mcp_servers=[server])
    
    with trace('conversation'):
        result = await Runner.run(agent, question)
    
    display(Markdown(result.final_output))

You are running a workshop about AI Agents and co-presenting it with Jon.

## Server 2 - Internet Search (Google Custom Search API)

Check the instructions from <a href="https://github.com/limklister/mcp-google-custom-search-server">here.</a>

For the API Key (1st prerequisite in the README):
- Enable Custom Search API in one of your projects from your Google Console
- Go to `APIs and Services` > `Credentials` to create an API Key (it's not under `APIs and Services` > `Custom Search` > `Credentials`)
  - Make sure to check `Custom Search` in the APIs

Make sure there are no spaces in the path when running `node path/to/build/index.js`

In [23]:
params = {
    "command": "node",
    "args": [f"{os.getcwd()}/mcp-google-custom-search-server/build/index.js"],
    "env": {"GOOGLE_API_KEY": os.environ['GOOGLE_API_KEY'], "GOOGLE_SEARCH_ENGINE_ID": os.environ['GOOGLE_SEARCH_ENGINE_ID']}
}

async with MCPServerStdio(params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='search', description='Search the web using Google Custom Search API', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query'}, 'numResults': {'type': 'number', 'description': 'Number of results to return (max 10)', 'default': 5}}, 'required': ['query']}, annotations=None)]

In [34]:
instructions = "You are able to search the web for information and briefly summarize the takeaways. Use the search tool you have been provided. If there are any issues, then your response must include them."
request = f"Please research the latest news on Amazon stock price and summarize its outlook. For context, it is {datetime.now().strftime('%Y-%m-%d')}"

In [35]:
async with MCPServerStdio(params, client_session_timeout_seconds=60) as server:
    agent = Agent(name='agent', instructions=instructions, model=model, mcp_servers=[server])

    with trace('conversation'):
        result = await Runner.run(agent, request)
    
    display(Markdown(result.final_output))

As of August 5, 2025, Amazon (AMZN) stock is trading around $214.75, with some data indicating a slight increase on the day. Over the past year, the stock's performance has been positive, though it has underperformed the S&P 500 in a five-year return comparison.

Morningstar recently highlighted strong first-quarter 2025 earnings for Amazon, with Amazon Web Services (AWS) continuing to be a key driver. While there were some concerns about profitability guidance, Morningstar views these fears as overblown.

Looking ahead, analysts have an average price target of $265.16 for AMZN, with a high forecast of $305.00 and a low of $235.00. This suggests a potential upside of approximately 23.47% from the current price. Investors are actively watching the stock, with some sources indicating it might be a good time to consider buying.

## Server 3 - Live Market Data (Polygon.io)

In [14]:
params = {
    "command": "uvx",
    "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"],
    "env": {"POLYGON_API_KEY": os.environ['POLYGON_API_KEY']}
}

async with MCPServerStdio(params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='get_aggs', description='\nList aggregate bars for a ticker over a given date range in custom time window sizes.\n', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}, 'multiplier': {'title': 'Multiplier', 'type': 'integer'}, 'timespan': {'title': 'Timespan', 'type': 'string'}, 'from_': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'From'}, 'to': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'To'}, 'adjusted': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': None, 'title': 'Adjusted'}, 'sort': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Sort'}, 'limit': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Limit'}, 'params': {'anyOf': [{'additionalProperties': True, 'type': 'object'}, {'typ

In [ ]:
instructions = "You answer questions about the stock market."
# add constraint to use get_daily_open_close_agg tool
# can't use get_snapshot_ticker on free plan
request = "What's the share price of Apple? Use your get_daily_open_close_agg tool to get the latest price."

async with MCPServerStdio(params, client_session_timeout_seconds=60) as server:
    agent = Agent(name='agent', instructions=instructions, model=model, mcp_servers=[server])

    with trace('conversation'):
        result = await Runner.run(agent, request)
    
    display(Markdown(result.final_output))

On April 23, 2024, Apple (AAPL) closed at 166.9.